<a href="https://colab.research.google.com/github/Teja3993/Algorithms-and-Analysis-Lab/blob/main/Algorithms_Lab_24th_September.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from collections import deque

INF = float('inf')  # A large number representing infinite distance
NIL = -1  # Represents no pair or unmatched node

def bfs(adj, pair_u, pair_v, dist, m):
    """
    Breadth-first search to build layers (levels) of the graph and find
    all shortest augmenting paths simultaneously.

    Args:
        adj: adjacency list representing edges from U to V.
        pair_u: current pair for each node in U (index corresponds to U nodes).
        pair_v: current pair for each node in V.
        dist: distances (layers) for nodes in U.
        m: number of nodes in U.

    Returns:
        True if there is at least one augmenting path (shortest path)
        from an unmatched node in U to an unmatched node in V.
    """
    queue = deque()
    # Initialize distances: 0 for unmatched U-nodes, INF otherwise
    for u in range(m):
        if pair_u[u] == NIL:
            dist[u] = 0
            queue.append(u)
        else:
            dist[u] = INF

    found_augmenting_path = False

    # Standard BFS loop to set distances (levels) and find if augmenting path exists
    while queue:
        u = queue.popleft()
        for v in adj[u]:
            # If v is free (unmatched), we found an augmenting path
            if pair_v[v] == NIL:
                found_augmenting_path = True
            # If v is matched, and the pair node in U has not been visited yet
            elif dist[pair_v[v]] == INF:
                dist[pair_v[v]] = dist[u] + 1
                queue.append(pair_v[v])

    return found_augmenting_path

def dfs(u, adj, pair_u, pair_v, dist):
    """
    Depth-first search to find augmenting paths using the layering from BFS.

    Args:
        u: current node in U being explored.
        adj: adjacency list.
        pair_u: current pairing for nodes in U.
        pair_v: current pairing for nodes in V.
        dist: distances set by BFS.

    Returns:
        True if an augmenting path is found starting from u.
    """
    for v in adj[u]:
        # Condition 1: v is unmatched
        stmt1 = (pair_v[v] == NIL)
        # Condition 2: v is matched, but we can recurse on its pair in U if it's on the next layer
        stmt2 = (dist[pair_v[v]] == dist[u] + 1 and dfs(pair_v[v], adj, pair_u, pair_v, dist))

        if stmt1 or stmt2:
            # If either condition is true, we can augment along this edge
            pair_u[u] = v
            pair_v[v] = u
            return True

    # Mark u as unreachable in this layering
    dist[u] = INF
    return False

def hopcroft_karp(m, n, edges):
    """
    Main function to compute maximum bipartite matching using Hopcroft-Karp algorithm.

    Args:
        m: number of vertices on the left side (U).
        n: number of vertices on the right side (V).
        edges: list of edges (u, v) from U to V.

    Returns:
        The size of the maximum matching.
    """
    # Build adjacency list from U to V
    adj = [[] for _ in range(m)]
    for u, v in edges:
        adj[u].append(v)

    # pair_u[u] = v means u in U is matched to v in V, NIL if unmatched
    pair_u = [NIL] * m
    # pair_v[v] = u means v in V is matched to u in U, NIL if unmatched
    pair_v = [NIL] * n
    dist = [INF] * m  # Distance labels for BFS layering

    matching = 0  # Count of matched pairs

    # Repeat until no more augmenting paths are found
    while bfs(adj, pair_u, pair_v, dist, m):
        # Try to find augmenting paths starting from unmatched vertices in U
        for u in range(m):
            if pair_u[u] == NIL:
                if dfs(u, adj, pair_u, pair_v, dist):
                    matching += 1  # Increase matching size for each augmenting path found

    return matching

# Example usage
m, n = 3, 3
edges = [
    (0, 0), # U1 to V1
    (0, 1), # U1 to V2
    (1, 0), # U2 to V1
    (2, 2), # U3 to V3
]

print("Maximum matching size:", hopcroft_karp(m, n, edges))


Maximum matching size: 3
